In [1]:
import pandas as pd

# 读取父文件夹下day13的FBlocation的数据
data = pd.read_csv("../day13/FBlocation/train.csv")

print(data.shape)
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")
print(data.shape)

y = data['place_id']

# 删除不需要的列，保留特征列
X = data.drop(['row_id', 'place_id'], axis=1)

# 处理time列，转换为datetime格式并提取时间特征
X['time'] = pd.to_datetime(X['time'], unit='s')
X['day'] = X['time'].dt.day  # 提取日期中的天数
X['hour'] = X['time'].dt.hour  # 提取时间中的小时
X['weekday'] = X['time'].dt.weekday  # 提取星期几（0=周一，6=周日）
X = X.drop(['time'], axis=1)

print("特征矩阵形状：", X.shape,type(X))
print("标签向量形状：", y.shape,type(y))
print("特征列：", X.columns.tolist())
print("前5行特征数据：")
print(X.head())

(29118021, 6)
(17710, 6)
特征矩阵形状： (17710, 6) <class 'pandas.core.frame.DataFrame'>
标签向量形状： (17710,) <class 'pandas.core.series.Series'>
特征列： ['x', 'y', 'accuracy', 'day', 'hour', 'weekday']
前5行特征数据：
           x       y  accuracy  day  hour  weekday
600   1.2214  2.7023        17    1    18        3
957   1.1832  2.6891        58   10     2        5
4345  1.1935  2.6550        11    5    15        0
4735  1.1452  2.6074        49    6    23        1
5580  1.0089  2.7287        19    9    11        4


In [2]:
unique_places = y.nunique()

# 统计每个place_id的重复数目
place_counts = y.value_counts()
print(f"\n各place_id的重复数目：")
print(place_counts)
# 显示重复数目的统计信息
print(f"\n重复数目统计：")
print(f"最多重复次数：{place_counts.max()}")
print(f"最少重复次数：{place_counts.min()}")
print(f"平均重复次数：{place_counts.mean():.2f}")



各place_id的重复数目：
place_id
1097200869    1044
6683426742     894
3312463746     889
4932578245     863
5606572086     656
              ... 
1211672581       1
2305788054       1
4506029517       1
5843601955       1
9076695703       1
Name: count, Length: 805, dtype: int64

重复数目统计：
最多重复次数：1044
最少重复次数：1
平均重复次数：22.00


In [3]:
places_to_keep = place_counts[place_counts > 2].index

mask = y.isin(places_to_keep)
X_filtered = X[mask]
y_filtered = y[mask]

print(f"过滤前样本数量：{len(X)}")
print(f"过滤后样本数量：{len(X_filtered)}")
print(f"过滤前place_id数量：{y.nunique()}")
print(f"过滤后place_id数量：{y_filtered.nunique()}")
print(len(y))

# 更新X和y
X = X_filtered
y = y_filtered

print(f"\n过滤后的数据形状：")
print(f"特征矩阵形状：{X.shape}")
print(f"标签向量形状：{y.shape}")

过滤前样本数量：17710
过滤后样本数量：17086
过滤前place_id数量：805
过滤后place_id数量：295
17710

过滤后的数据形状：
特征矩阵形状：(17086, 6)
标签向量形状：(17086,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# 划分训练集和测试集，测试集占15%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

print(f"训练集样本数量：{len(X_train)}")
print(f"测试集样本数量：{len(X_test)}")
print(f"训练集place_id数量：{y_train.nunique()}")
print(f"测试集place_id数量：{y_test.nunique()}")

# 数据标准化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  #fit_transform会在训练集计算均值和方差
X_test_scaled = scaler.transform(X_test)   #不会再次计算均值和方差，而是直接进行标准化

# 创建KNN分类器
knn = KNeighborsClassifier(n_neighbors=9,weights='distance')

# 训练模型
print("\n开始训练KNN模型...")
knn.fit(X_train_scaled, y_train)

# 计算准确率
accuracy = knn.score(X_test_scaled, y_test)
print(f"\nKNN模型准确率：{accuracy:.4f}")

训练集样本数量：15377
测试集样本数量：1709
训练集place_id数量：295
测试集place_id数量：208

开始训练KNN模型...

KNN模型准确率：0.4991


In [ ]:
# 使用网格搜索找到最佳的n_neighbors参数
from sklearn.model_selection import GridSearchCV

# 定义参数网格
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'] #uniform: 每个邻居的权重相同 distance: 根据距离进行加权，距离越近权重越大，距离越远权重越小
}

# 创建KNN分类器
knn_grid = KNeighborsClassifier()

# 创建网格搜索对象，使用3折交叉验证
grid_search = GridSearchCV(
    estimator=knn_grid,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 执行网格搜索
print("开始网格搜索...")
grid_search.fit(X_train_scaled, y_train)

# 输出最佳参数和最佳得分
print(f"\n最佳参数：{grid_search.best_params_}")
print(f"最佳交叉验证得分：{grid_search.best_score_:.4f}")

# 使用最佳参数在测试集上评估
best_knn = grid_search.best_estimator_
test_accuracy = best_knn.score(X_test_scaled, y_test)
print(f"最佳模型在测试集上的准确率：{test_accuracy:.4f}")


开始网格搜索...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

最佳参数：{'n_neighbors': 9, 'weights': 'distance'}
最佳交叉验证得分：0.4874
最佳模型在测试集上的准确率：0.4991
